In [ ]:
import sys
sys.executable = '/home/sirui/miniconda3/envs/myenv/bin/python'
sys.path.insert(0, '/home/sirui/miniconda3/envs/myenv/lib/python3.8/site-packages')
import torch
import math
import random
import scipy.io
import numpy as np
import torch


import torch.nn.functional as F

import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.data import Data,InMemoryDataset, download_url
from typing import Optional, Callable, List, Union, Tuple, Dict, Iterable
from torch_geometric.nn import GCNConv

from scipy.spatial import distance
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg as sla

from matplotlib import pyplot as plt
from matplotlib import image as img
import copy
import torch.nn as nn
from torch_geometric.loader import DataLoader
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import GridSearchCV
from tqdm import trange
from sklearn.model_selection import StratifiedKFold
from torch_geometric.utils import subgraph
from Graph_Construction import adjacency_matrix, get_dataset, MyOwnDataset
from Augmentations import UniformSample, EdgePerturbation, NodeAttrMask, RWSample, random
from contrasive import Contrastive
from GraphCL import GraphCL, Encoder, GCN, GraphUnsupervised, NodeUnsupervised
from Superpixel import return_from_sp,return_from_sp_bool, process_superpixel
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
import os
# Assuming VQVAE class and other components are already defined
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from datetime import datetime
project_root = "Indian_Pines/"   ### Please change the root!!!
def load_data(project_root):
    
    mat = scipy.io.loadmat(project_root+'Indian_pines.mat')
    IP_dataset=mat["indian_pines"]/10000

    mat = scipy.io.loadmat(project_root+'Indian_pines_gt.mat')
    IP_gt=mat["indian_pines_gt"]

    ch_num = np.size(IP_dataset,2)
    band_list = np.zeros(ch_num-20)
    for i in range(0,103):
          band_list[i]=i
    for i in range(108,149):
          band_list[i-5]=i
    for i in range(163,219):
          band_list[i-19]=i
    band_list=np.array(band_list,dtype=int)
    IP_dataset=IP_dataset[:,:,band_list]
    
    Dim=np.array([np.size(IP_dataset,0),np.size(IP_dataset,1),np.size(IP_dataset,2)])
    bands_num = Dim[2]
    bands_xy_num = Dim[2]+2

    num_classes=np.max(IP_gt)

    y_c, x_c = np.meshgrid(np.arange(Dim[0]), np.arange(Dim[1]))
    cor=np.dstack((x_c,y_c))/144
    IP_dataset=np.concatenate((IP_dataset, cor), axis=2)

    data = np.reshape(IP_dataset,(Dim[0]*Dim[1],bands_xy_num))
    gt_flat=np.reshape(IP_gt,(Dim[0]*Dim[1],1))
    newDim=np.size(data,0)
    return IP_dataset,IP_gt,Dim,bands_num,bands_xy_num,num_classes,data,gt_flat,newDim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
IP_dataset,IP_gt,Dim,bands_num,bands_xy_num,num_classes,data,gt_flat,newDim=load_data(project_root)
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
colorList = ["#000000","#601d97","#00d300", "#87ff00","#2d9662","#387a41","#ac5c2e","#ff8a54","#00ffff","#915085","#ffffff","#dec6de","#ff0000","#980000","#0071ff","#ffff00","#f4a300"]

my_cmap = ListedColormap(colorList, name="my_cmap")

In [ ]:
def get_oa_perclass(gt, pred):
    oa_perclass = []
    for i in range(1, 17):
        oa_perclass.append(accuracy_score(gt[gt == i], pred[gt == i]))
    return oa_perclass

In [ ]:
def experiment(project_root, config):
    num_epochs,Second_gen,n_layers,p_lr,num_labels,num_seg,sigma,WeightMode,xy_scaler,lamda=config.num_epochs,config.Second_gen,config.n_layers,config.p_lr,config.num_labels,config.num_seg,config.sigma,config.WeightMode,config.xy_scaler,config.lamda
    IP_dataset,IP_gt,Dim,bands_num,bands_xy_num,num_classes,data,gt_flat,newDim=load_data(project_root)
    train_m=np.load(project_root+"train_mask_"+str(num_labels)+".npy",allow_pickle=True)
    data,gt_flat,train_m,slic_graph=process_superpixel(IP_dataset,IP_gt,train_m,num_seg,num_labels,Dim,project_root,'mean')
    def options(mode,k,sigma,miu):
      options = {
                "NeighborMode": 'KNN',
                "k": k,
                "WeightMode": mode,
                "t":sigma**2,
                "xy_scaler":miu,
                }
      return options
    options=options(WeightMode,10,sigma,xy_scaler)
    L=adjacency_matrix(data,options)
    data_=get_dataset(data,gt_flat,L,train_m,None,options)
    print(data_)
    dataset=MyOwnDataset("",data_,Second_gen=Second_gen)
    dataset.process()
    np.shape(np.where(dataset.data.train_mask == True))
    embed_dim = 1024
    encoder = Encoder(feat_dim=dataset.data.x.shape[1], hidden_dim=embed_dim, 
                  n_layers=n_layers,pool='sum',node_level=False, graph_level=True)
    GraphCL_m = GraphCL(dim=embed_dim, aug_1=config.aug1,aug_2=config.aug2, aug_ratio=config.aug_ratio, lamda=config.lamda)
    evaluator = GraphUnsupervised(dataset,classifier=config.classifier,device=device, log_interval=1
                              )
    evaluator.setup_train_config(batch_size = 128, p_lr=p_lr, p_epoch=num_epochs, p_weight_decay=1e-5)
    acc, sd ,idx,result, test_scores_m,train_scores=evaluator.evaluate(learning_model=GraphCL_m, encoder=encoder)
    result= return_from_sp(slic_graph,result,Dim)
    print(train_scores)
    np.save((project_root+"output/result/graphcl_"+str(Second_gen)+str(n_layers)+str(num_labels)+str(num_seg)+str(sigma)+str(options["WeightMode"])+str(options["xy_scaler"])+str(config.lamda)+str(config.aug1)+str(config.aug2)+"s.npy"), result)
    print('Best epoch %d: acc %.4f +/-(%.9f)'%((idx+1), acc, sd))
    fig= plt.figure(figsize=(5, 4))   
    plt.plot(test_scores_m)
    plt.title('Best epoch %d: acc %.4f +/-(%.9f)'%((idx+1), acc, sd))
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Test'], loc='upper left')
    plt.savefig(project_root+'output/plots/plot_graphcl_'+str(Second_gen)+str(n_layers)+str(num_labels)+str(num_seg)+str(sigma)+str(options["WeightMode"])+str(options["xy_scaler"])+str(config.lamda)+str(config.aug1)+str(config.aug2)+'s.png')
    plt.show()
    print(result.max())
    result[0]=16
    result[1]=0
    fig = plt.figure(figsize=(20, 20))

    ax1 = fig.add_subplot(2,2,1)
    ax1=plt.imshow(np.reshape(result,[Dim[0],Dim[1]]),cmap=my_cmap)
    plt.title("predict")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    ax2 = fig.add_subplot(2,2,2)
    o_train_m=return_from_sp_bool(slic_graph,train_m,Dim)
    ax2=plt.imshow(np.reshape(o_train_m,[Dim[0],Dim[1]]),cmap=my_cmap)
    plt.title("Trianing Image")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    ax3 = fig.add_subplot(2,2,3)
    ax3=plt.imshow(IP_gt,cmap=my_cmap)
    plt.title("Ground Truth")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    ax2 = fig.add_subplot(2,2,4)
    maskimage=np.reshape(result,[Dim[0],Dim[1]])
    maskimage[np.where(IP_gt==0)]=0
    maskimage=np.reshape(np.array(maskimage),[Dim[0],Dim[1]])
    ax2=plt.imshow(maskimage,cmap=my_cmap)
    plt.title("Mask Image")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")
    plt.savefig(project_root+'output/plots/fig_graphcl_'+str(Second_gen)+str(n_layers)+str(num_labels)+str(num_seg)+str(sigma)+str(options["WeightMode"])+str(options["xy_scaler"])+str(config.lamda)+str(config.aug1)+str(config.aug2)+'s.png')
    plt.show()
    o_train_m=np.load(project_root+"train_mask_"+str(num_labels)+".npy",allow_pickle=True)
    tr=np.reshape(o_train_m,[Dim[0],Dim[1]])
    gt=IP_gt
    gt[np.where(tr==True)]=0
    test_acc = accuracy_score(IP_gt[np.where(gt!=0)], maskimage[np.where(gt!=0)])
    print(test_acc)

    pred=maskimage[np.where(gt!=0)]
    gt=IP_gt[np.where(gt!=0)]
    plt.show()
    conf_matrix=confusion_matrix(gt, pred)
    acc_score=accuracy_score(gt, pred)
    cohen_kappa=cohen_kappa_score(gt, pred)
    class_report=classification_report(gt, pred)
    oa_perclass=get_oa_perclass(gt, pred)
    print(conf_matrix,"OA:",acc_score,"Kappa:",cohen_kappa,class_report,oa_perclass)

    # Saving the results to a text file
    outputfile = f"""
    START!Num_epochs,Second_gen,n_layers,p_lr,num_labels,num_seg,sigma,WeightMode,xy_scaler:{ num_epochs,Second_gen,n_layers,p_lr,num_labels,num_seg,sigma,WeightMode,xy_scaler}
    aug1,aug2,ratio,lamda,classifier:{config.aug1,config.aug2, config.aug_ratio,config.lamda,config.classifier}
    Accuracy Score: {test_acc}
    Confusion Matrix:
    {conf_matrix}
    Accuracy Score (non-zero gt): {acc_score}
    Cohen Kappa Score: {cohen_kappa}
    Classification Report:
    {class_report}
    OA Per Class:
    {oa_perclass}
    """
    return outputfile


In [ ]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path=project_root+'output/txt/result_'+str(current_time)+'.txt'

class TrainingConfig:
    num_epochs=400
    Second_gen=6
    n_layers=8
    p_lr=0.0000000005
    num_labels = 2000
    num_seg=9000
    sigma=1
    WeightMode="Binary"
    xy_scaler=0.6
    aug1="strong"
    aug2="weak"
    aug_ratio=0.4
    lamda=0.83
    classifier='LogReg'
config = TrainingConfig()
results=[]
for num_labels in [10,20,30,40,50,2000]:
    config.num_labels=num_labels
    result=experiment(project_root, config)
    results.append(result)
with open(file_path, 'w') as file:
    for i in range(0,len(results)):
        file.write(results[i])